# 计算两维的xy的平方根得到一维
*  `feature1 =  torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根`

In [10]:
import torch

c = torch.tensor([[3,4,5],[6,8,4]], dtype=torch.float)
print(c)

tensor([[3., 4., 5.],
        [6., 8., 4.]])


In [11]:
feature1 =  torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根
# feature1 =  c[:, 0] + c[:,1]
feature1 = feature1.type_as(c).view(-1, 1) # 维度要一致 (3, 1)
print(feature1)
print()

tensor([[ 5.],
        [10.]])



In [12]:
# c[:, 1] = feature1.squeeze()
c[:, 1] = torch.sqrt( c[:, 0]**2 +  c[:,1]**2) # 两维的平方根
print(c[:, 1:])

tensor([[ 5.,  5.],
        [10.,  4.]])


In [13]:
feature = [c]
feature.append(feature1)
print(feature) # 列表
feature = torch.cat(feature, dim=1) #  最后一维合并Combine together feature decorations
print(feature)

[tensor([[ 3.,  5.,  5.],
        [ 6., 10.,  4.]]), tensor([[ 5.],
        [10.]])]
tensor([[ 3.,  5.,  5.,  5.],
        [ 6., 10.,  4., 10.]])
